# A GENTLE INTRODUCTION TO TORCH.AUTOGRAD

Notebook Conversion / Exercises: Sebastian Klaßmann, University of Cologne  
February $22^{nd}$, 2021

**Disclaimer**: The text and core structure of this series of notebooks has been taken from the above link. The provided materials have been converted to Jupyter notebooks and comprehension questions and exercises have been added along the way.

---

In [ ]:
import numpy as np

**Task 1**: Read through the below Notebook carefully and discuss with your fellow students along the way.  
**Task 2**: Throughout this notebook, there will be extra questions and programming challenges for you. Solve them - if you are unsure about how certain things work in pytorch or numpy, the library documentations are your best friends.  
**Task 3**: Often enough, the amount of commenting in code that you can find online is not sufficient to enable somebody new to a given library / programming language to understand what is going on. Complete the comments in the code below and make sure that you understand every line of code.  
   
**Tip:** Very often, libraries in Python come with a built in documentation for all included object types and methods. You can easily access them using a question mark before the object in question. For example:

In [ ]:
# ?np.array
# ?np.mat

---

```torch.autograd``` is PyTorch’s automatic differentiation engine that powers neural network training. In this section, you will get a conceptual understanding of how autograd helps a neural network train.

### Background

Neural networks (NNs) are a collection of nested functions that are executed on some input data. These functions are defined by parameters (consisting of weights and biases), which in PyTorch are stored in tensors.

Training a NN happens in two steps:

**Forward Propagation**: In forward prop, the NN makes its best guess about the correct output. It runs the input data through each of its functions to make this guess.

**Backward Propagation**: In backprop, the NN adjusts its parameters proportionate to the error in its guess. It does this by traversing backwards from the output, collecting the derivatives of the error with respect to the parameters of the functions (gradients), and optimizing the parameters using gradient descent. For a more detailed walkthrough of backprop, check out this video from 3Blue1Brown:

In [ ]:
from IPython.display import YouTubeVideo

In [ ]:
# here's the video in case you want to check it out at your own leasure.
YouTubeVideo('tIeHLnjs5U8', width=800, height=300)

**Literature:** A more general approach to understanding backpropagation in neural networks based on understanding neural nets as directed acyclic graphs is given in Rojas, R. (2013). *Neural networks: a systematic introduction.* Springer Science & Business Media, 151-173. 

## Usage in PyTorch

Let’s take a look at a single training step. For this example, we load a pretrained resnet18 model from torchvision. We create a random data tensor to represent a single image with 3 channels, and height & width of 64, and its corresponding label initialized to some random values.

In [ ]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

---

### Exercise: 
  
Look at the data we created above.  
  
```python
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)
```
* What does our resulting label tensor look like?  

Have a look at Pytorch's [description of the (pretrained) resnet18 architecture](https://pytorch.org/hub/pytorch_vision_resnet/).  
* What does it mean to represent an image "with 3 channels" and why could it be useful?

* What does the structure of our pretrained network look like? If you make use of external references, write them down.
  
* **Bonus Questions:** In image processing, one often used architecture is a convolutional neural network (ConvNet/CNN). [This article](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) might prove to be a good starting point.
    * What is the core idea behind a convolutional layer and what are the differences between max pooling and average pooling?
    * In your own words: Why is Convolution feasible when dealing with multi-channel image input data represented as tensors?

___


Next, we run the input data through the model through each of its layers to make a prediction. This is the forward pass.

In [ ]:
prediction = model(data) # forward pass

We use the model’s prediction and the corresponding label to calculate the error (loss). The next step is to backpropagate this error through the network. Backward propagation is kicked off when we call .backward() on the error tensor. Autograd then calculates and stores the gradients for each model parameter in the parameter’s .grad attribute.

In [ ]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

Next, we load an optimizer, in this case SGD with a learning rate of 0.01 and momentum of 0.9. We register all the parameters of the model in the optimizer.

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Finally, we call ```.step()``` to initiate gradient descent. The optimizer adjusts each parameter by its gradient stored in ```.grad```.

In [ ]:
optim.step() #gradient descent

At this point, you have everything you need to train your neural network. The below sections detail the workings of autograd - feel free to skip them.

---

### Exercise:
* provide full comments for the above code cells.
  
One of the major strengths of PyTorch is the explicitness regarding individual steps of the training process. Using your knowledge from our previous sessions, please:
  
* instantiate new copys of the pretrained resnet18 architecture and our mock data.  
* change the loss function to accomodate the sum of squared errors (SSE) we have been using throughout our course.
* change the learning rate to 0.1
* Perform a single step of gradient descent.

---

### Differentiation in Autograd

Let’s take a look at how ``` autograd ``` collects gradients. We create two tensors a and b with ``` requires_grad=True ```. This signals to ``` autograd ``` that every operation on them should be tracked.

In [ ]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

We create another tensor Q from a and b.

$$Q = 3a^3 - b^2$$

In [ ]:
Q = 3*a**3 - b**2

Let’s assume a and b to be parameters of an NN, and Q to be the error. In NN training, we want gradients of the error with regards to parameters, i.e.

$$ \frac {\partial Q}{\partial a} = 9a^2$$  
  
$$ \frac {\partial Q} {\partial b} = -2b$$

When we call ```.backward()``` on Q, autograd calculates these gradients and stores them in the respective tensors’ ```.grad ``` attribute.

We need to explicitly pass a gradient argument in ``` Q.backward() ``` because it is a vector. ``` gradient ``` is a tensor of the same shape as Q, and it represents the gradient of Q w.r.t. itself, i.e.  
  
$$ \frac{dQ}{dQ} = 1 $$

Equivalently, we can also aggregate Q into a scalar and call ```backward ``` implicitly, like ```Q.sum().backward()```.

In [ ]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Gradients are now deposited in ```a.grad ``` and ```b.grad```

In [ ]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

---

### Exercises:  
  
* Using your knowledge about derivation rules, show on paper that the partial derivations of Q provided in the section above are correct.  
  
* Consider a different loss function, e.g., $ L(a,b) = 5a^3 - 6b^2$.  
   
* Calculate the partial derivatives of L with regards to $\vec a$ and $\vec b$. 
  
* Create two new tensors a and b representing these vectors, signalling to ```autograd``` that gradients for these tensors should be tracked down the line:
  
$$
\vec a = 
\begin{bmatrix}
8 & 9 & 24 & 13 \\
\end{bmatrix}
\vec b = 
\begin{bmatrix}
17 & 29 & 2 & 0.7 \\
\end{bmatrix}
$$  
  
* Define a gradient tensor representing the partial derivative $\frac{\partial Q}{\partial Q}$ in an appropriate shape.  
* Check if the collected gradients match your own partial derivations $\frac{\partial Q}{\partial a}$ and $\frac{\partial Q}{\partial b}$.

---

### Optional Reading - Vector Calculus using ```autograd```



Mathematically, if you have a vector valued function $\vec y = f(\vec x)$, then the gradient of $\vec y$ with respect to $\vec x$ is a Jacobian matrix $J$:



$$ 
J = (\frac{\partial y}{\partial x_1} , ... ,  \frac{\partial y}{\partial x_n}) 
=
\begin{pmatrix}
\frac{\partial y_1}{\partial x_1} & \dots & \frac{\partial y_1}{\partial x_n} \\ 
\vdots & \ddots & \vdots \\ 
\frac{\partial y_m}{\partial x_1} & \dots & \frac{\partial y_m}{\partial x_n}
\end{pmatrix}
$$

Generally speaking, ```torch.autograd``` is an engine for computing vector-Jacobian product. That is, given any vector $\vec v$, compute the product $J^T \cdot \vec v$ .

If $v$ happens to be the gradient of a scalar function

$$
l = g(\vec y) = (\frac{\partial l}{\partial y_1} \dots \frac{\partial l}{\partial y_m})^T
$$

then by the chain rule, the vector-Jacobian product would be the gradient of $l$
with respect to $\vec x$: 

$$
J^T \cdot \vec v = 
\begin{pmatrix}
\frac{\partial y_1}{\partial x_1} & \dots & \frac{\partial y_m}{\partial x_1} \\ 
\vdots & \ddots & \vdots \\ 
\frac{\partial y_1}{\partial x_n} & \dots & \frac{\partial y_m}{\partial x_n}
\end{pmatrix}
\begin{pmatrix}
\frac{\partial l}{\partial y_1} \\
\dots \\
\frac{\partial l}{\partial y_n}
\end{pmatrix}
=
\begin{pmatrix}
\frac{\partial l}{\partial x_1} \\
\dots \\
\frac{\partial l}{\partial x_n}
\end{pmatrix}
$$

This characteristic of vector-Jacobian product is what we use in the above example; ```external_grad ``` represents $\vec v$.

---

### Computational Graph

Conceptually, autograd keeps a record of data (tensors) & all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function) objects. In this DAG, leaves are the input tensors, roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

* run the requested operation to compute a resulting tensor, and
* maintain the operation’s gradient function in the DAG.  
  
The backward pass kicks off when ```.backward()``` is called on the DAG root. ```autograd ``` then:

* computes the gradients from each ```.grad_fn ```,
* accumulates them in the respective tensor’s ```.grad ``` attribute, and
* using the chain rule, propagates all the way to the leaf tensors.  
  
Below is a visual representation of the DAG in our example. In the graph, the arrows are in the direction of the forward pass. The nodes represent the backward functions of each operation in the forward pass. The leaf nodes in blue represent our leaf tensors a and b.

<img src='https://pytorch.org/tutorials/_images/dag_autograd.png' width=400>

---
**NOTE**  
DAGs are dynamic in PyTorch An important thing to note is that the graph is recreated from scratch; after each ```.backward() ```call, autograd starts populating a new graph. This is exactly what allows you to use control flow statements in your model; you can change the shape, size and operations at every iteration if needed.  
  
___

### Exclusion from the DAG

```torch.autograd ``` tracks operations on all tensors which have their ```requires_grad ``` flag set to True. For tensors that don’t require gradients, setting this attribute to False excludes it from the gradient computation DAG.

The output tensor of an operation will require gradients even if only a single input tensor has ```requires_grad=True ```.

In [ ]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

In a NN, parameters that don’t compute gradients are usually called **frozen parameters**. It is useful to “freeze” part of your model if you know in advance that you won’t need the gradients of those parameters (this offers some performance benefits by reducing autograd computations).

Another common usecase where exclusion from the DAG is important is for [finetuning a pretrained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)

In finetuning, we freeze most of the model and typically only modify the classifier layers to make predictions on new labels. Let’s walk through a small example to demonstrate this. As before, we load a pretrained resnet18 model, and freeze all the parameters.

In [ ]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

Let’s say we want to finetune the model on a new dataset with 10 labels. In resnet, the classifier is the last linear layer ```model.fc```. We can simply replace it with a new linear layer (unfrozen by default) that acts as our classifier.

In [ ]:
model.fc = nn.Linear(512, 10)

Now all parameters in the model, except the parameters of ```model.fc ```, are frozen. The only parameters that compute gradients are the weights and bias of ```model.fc```.

In [ ]:
# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

Notice although we register all the parameters in the optimizer, the only parameters that are computing gradients (and hence updated in gradient descent) are the weights and bias of the classifier.

The same exclusionary functionality is available as a context manager in [torch.no_grad()](https://pytorch.org/docs/stable/generated/torch.no_grad.html).

---

### Further readings:

* [In-place operations & Multithreaded Autograd](https://pytorch.org/docs/stable/notes/autograd.html)  
* [Example implementation of reverse-mode autodiff](https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC)